# Identificação de motoristas através de comportamentos de condução

##  Imports

In [1]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

## Gerando dados sintéticos de comportamentos de condução para 10 motoristas

In [2]:
# Configurações
num_motoristas = 10
segundos_por_motorista = 86400  # 1 dia de dados por motorista
total_linhas = num_motoristas * segundos_por_motorista

# Função para gerar dados sintéticos para um motorista
def gerar_dados_motorista(motorista_id, n_registros):
    np.random.seed(motorista_id)  # manter reprodutibilidade por motorista

    # Simula variações realistas por motorista
    velocidade_base = random.uniform(40, 100)
    aceleracao_base = random.uniform(-1, 1)
    rotacao_base = random.uniform(1000, 3000)
    consumo_base = random.uniform(8, 15)

    dados = {
        'motorista_id': [motorista_id] * n_registros,
        'timestamp': pd.date_range("2025-04-10 00:00:00", periods=n_registros, freq='S'),
        'velocidade_kmh': np.clip(np.random.normal(velocidade_base, 10, n_registros), 0, 180),
        'aceleracao_ms2': np.clip(np.random.normal(aceleracao_base, 0.5, n_registros), -3, 3),
        'rpm': np.clip(np.random.normal(rotacao_base, 300, n_registros), 500, 6000),
        'consumo_l_100km': np.clip(np.random.normal(consumo_base, 1.0, n_registros), 5, 25),
        'angulo_volante_graus': np.clip(np.random.normal(0, 30, n_registros), -180, 180),
        'posicao_pedal_acelerador': np.clip(np.random.normal(30, 20, n_registros), 0, 100),
        'freio_acionado': np.random.choice([0, 1], size=n_registros, p=[0.85, 0.15]),
        'marcha': np.random.choice(range(1, 7), size=n_registros),
        'gps_inclinacao': np.clip(np.random.normal(0, 3, n_registros), -10, 10),
        'gps_curva_graus': np.clip(np.random.normal(0, 20, n_registros), -180, 180)
    }

    return pd.DataFrame(dados)

## Salvando arquivo em .csv

## Lendo arquivo com dados sintéticos gerado

### Visualizando as primeiras linhas do conjunto de dados

In [3]:
df = pd.read_csv('../data/raw/dados_obd_sinteticos.csv')
df.head()

,motorista_id,timestamp,velocidade_kmh,aceleracao_ms2,rpm,consumo_l_100km,angulo_volante_graus,posicao_pedal_acelerador,freio_acionado,marcha,gps_inclinacao,gps_curva_graus
0,0,2025-04-10 00:00:00,87.641492,-1.164779,2856.580894,12.974922,-3.010104,49.875534,0,4,6.412683,19.300237
1,0,2025-04-10 00:00:01,74.002541,-0.248364,3077.701709,13.668439,33.277285,61.205832,0,1,2.937810,-11.193276
2,0,2025-04-10 00:00:02,79.788349,-0.921159,2830.124124,13.328252,-12.179117,7.030715,0,3,3.128054,5.762080
3,0,2025-04-10 00:00:03,92.409901,-0.880992,2645.434835,14.942527,48.682247,24.472256,0,6,-2.114854,7.320182
4,0,2025-04-10 00:00:04,88.676549,-0.618572,2639.916381,14.947222,30.107772,29.852586,0,5,-2.536785,-9.340282


## Análise Exploratória (EDA)

### Verificando colunas dos dados

In [4]:
df.columns

Index(['motorista_id', 'timestamp', 'velocidade_kmh', 'aceleracao_ms2', 'rpm',
       'consumo_l_100km', 'angulo_volante_graus', 'posicao_pedal_acelerador',
       'freio_acionado', 'marcha', 'gps_inclinacao', 'gps_curva_graus'],
      dtype='object')

### Verificando tamanho do conjunto de dados

In [5]:
df.shape

(864000, 12)

### Verificando a existência de dados nulos

In [6]:
df.isna().sum()

motorista_id                0
timestamp                   0
velocidade_kmh              0
aceleracao_ms2              0
rpm                         0
consumo_l_100km             0
angulo_volante_graus        0
posicao_pedal_acelerador    0
freio_acionado              0
marcha                      0
gps_inclinacao              0
gps_curva_graus             0
dtype: int64

### Verificando os tipos de dados de cada coluna

In [7]:
df.dtypes

motorista_id                  int64
timestamp                    object
velocidade_kmh              float64
aceleracao_ms2              float64
rpm                         float64
consumo_l_100km             float64
angulo_volante_graus        float64
posicao_pedal_acelerador    float64
freio_acionado                int64
marcha                        int64
gps_inclinacao              float64
gps_curva_graus             float64
dtype: object

### Verificando a quantidade de motoristas

In [8]:
df.motorista_id.unique()

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

### Realizando análise de distribuição das variáveis abaixo com uma amostra de dados para cada motorista: 

Para evitar sobrecarga de memória e visualizações densas demais devido ao alto volume de dados, optamos por utilizar **amostras aleatórias dos dados por motorista**. A análise a seguir mostra a distribuição das seguintes variáveis:

- `velocidade_kmh`  
- `aceleracao_ms2`  
- `rpm`  
- `consumo_l_100km`  
- `angulo_volante_graus`  
- `posicao_pedal_acelerador`  
- `marcha`  
- `gps_inclinacao`  
- `gps_curva_graus`  

As distribuições serão analisadas por meio de histogramas, gráficos de densidade (KDE) e boxplots. Essas abordagens facilitam a interpretação dos padrões de condução individuais e ajudam a identificar possíveis diferenças entre os motoristas com base em seu estilo de direção.


#### Amostra

In [10]:
def driver_samples(df, size):
    list_dataframes_samples = []
    for i in range(10):
        print('ID_Motorista:', i)
        df_driver = df[df.motorista_id == i]
        df_driver_sample = df_driver.sample(size, random_state=42)
        list_dataframes_samples.append(df_driver_sample)
    return pd.concat(list_dataframes_samples)

# def plots_analysis(amostra):
#     plt.figure(figsize=(10, 5))
#     sns.histplot(amostra['velocidade_kmh'], bins=50, kde=True)
#     plt.title('Histograma da Velocidade (km/h)')
#     plt.xlabel('Velocidade (km/h)')
#     plt.ylabel('Frequência')
#     plt.grid(True)
#     plt.show()

In [11]:
dv_samples = driver_samples(df, 10000)

ID_Motorista: 0
ID_Motorista: 1
ID_Motorista: 2
ID_Motorista: 3
ID_Motorista: 4
ID_Motorista: 5
ID_Motorista: 6
ID_Motorista: 7
ID_Motorista: 8
ID_Motorista: 9


In [12]:
dv_samples.head()

,motorista_id,timestamp,velocidade_kmh,aceleracao_ms2,rpm,consumo_l_100km,angulo_volante_graus,posicao_pedal_acelerador,freio_acionado,marcha,gps_inclinacao,gps_curva_graus
43404,0,2025-04-10 12:03:24,45.945584,-1.254292,3073.382390,13.554596,33.345751,25.230675,0,5,-4.624741,18.646467
78317,0,2025-04-10 21:45:17,63.047813,-0.486870,3159.726750,14.766736,31.056006,5.048942,0,5,1.594952,5.139402
6096,0,2025-04-10 01:41:36,76.780529,-0.628180,2982.387154,15.088523,-58.204085,0.000000,0,5,-0.036797,4.100237
40929,0,2025-04-10 11:22:09,73.633072,-0.680562,2862.234306,14.693674,13.893542,43.944134,0,3,-0.942629,17.686829
59505,0,2025-04-10 16:31:45,63.349937,-1.014227,2917.690892,15.495757,-1.488069,27.196232,0,6,-0.616065,-10.608583


#### Distribuição velocidade_kmh